### Notebook for prediction of stem cells population in adult healthy gut data from fetal data with `scANVI` 

- **Developed by:** Anna Maguza
- **Place:** Wuerzburg Institute for System Immunology
- **Created date:** 10th April 2024
- **Last modified date:** 16th April 2024

##### Import packages

In [ ]:
import scvi
import torch
import anndata
import warnings
import numpy as np
import scanpy as sc
import pandas as pd
import plotnine as p
from pywaffle import Waffle
import matplotlib.pyplot as plt
from scib_metrics.benchmark import Benchmarker

##### Setup working environment

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

In [ ]:
warnings.simplefilter(action = 'ignore')
scvi.settings.seed = 1712
%config InlineBackend.print_figure_kwargs = {'facecolor' : "w"}
%config InlineBackend.figure_format = 'retina'
torch.set_float32_matmul_precision('medium')

In [ ]:
arches_params = dict(
    use_layer_norm = "both",
    use_batch_norm = "none",
    encode_covariates = True,
    dropout_rate = 0.2,
    n_layers = 3,
)

In [ ]:
def X_is_raw(adata):
    return np.array_equal(adata.X.sum(axis=0).astype(int), adata.X.sum(axis=0))

### Read in data

+ Reference data (fetal stem cells)

In [ ]:
input = '/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/Fetal_stem_cells/Fetal_healthy_stem_cells_leiden.h5ad'
adata_ref = sc.read_h5ad(input)

In [ ]:
X_is_raw(adata_ref)

+ Query data (adult stem cells)

In [ ]:
adata_query = sc.read_h5ad('/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/Healthy_reference/Integrated/Integrated_4_datasets_05042024.h5ad')

In [ ]:
X_is_raw(adata_query)

#### Datasets preparation

+ Prepare obs in reference data

In [ ]:
adata_query2 = adata_query.copy()

In [ ]:
# filter out cells from adata_query2 that are not in adata_ref
adata_query2 = adata_query2[adata_query2.obs.index.isin(adata_ref.obs.index), :]

In [ ]:
adata_ref = anndata.AnnData(X = adata_query2.X, var = adata_query2.var, obs = adata_ref.obs)

In [ ]:
adata_ref.obs['seed_labels'] = adata_ref.obs['cluster'].copy()

+ Prepare query data

In [ ]:
adata_query = adata_query[adata_query.obs['Age_group'] == 'Adult', :]

In [ ]:
df = adata_query.obs['Cell States'].value_counts()

In [ ]:
stem_cells = ['Stem cells OLFM4', 'Stem cells OLFM4 LGR5', 'Stem cells OLFM4 PCNA', 'Stem_Cells_GCA', 'Stem cells OLFM4 GSTA1', 'Stem_Cells_ext']

adata_query = adata_query[adata_query.obs['Cell States'].isin(stem_cells), :]

In [ ]:
adata_query.obs['seed_labels'] = 'Unknown'

+ Concatenate datasets

In [ ]:
adata = adata_ref.concatenate(adata_query, batch_key = 'dataset', batch_categories = ['reference', 'query'], join='inner', index_unique = None)

In [ ]:
adata

In [ ]:
adata_raw = adata.copy()

#### Visualise uncorrected UMAP to identify batches

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

In [ ]:
sc.tl.umap(adata)

In [ ]:
adata.obs_keys

In [ ]:
sc.set_figure_params(dpi = 180, figsize=(10,10))
sc.pl.umap(adata, color=['Study_name','Age_group', 'Library_Preparation_Protocol', 'Donor_ID'], size=7)

In [ ]:
sc.set_figure_params(dpi = 180, figsize=(12,10))
sc.pl.umap(adata, color=['Region code', 'Fraction', 'Sex', 'Location'], size=7)

In [ ]:
sc.set_figure_params(dpi = 180, figsize=(12,10))
sc.pl.umap(adata, color=['n_genes_by_counts', 'total_counts', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo'], size=7)

#### Extract HGVs

In [ ]:
adata = adata_raw.copy()

In [ ]:
adata.layers['counts'] = adata.X.copy()

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    flavor = "seurat_v3",
    n_top_genes = 2000,
    layer = "counts",
    batch_key = "Donor_ID",
    subset = True,
    span = 1
)

#### Run scVI

In [ ]:
scvi.model.SCVI.setup_anndata(adata, 
                              categorical_covariate_keys=['Donor_ID', 'Library_Preparation_Protocol'],
                              labels_key = "seed_labels", 
                              layer = 'counts')

In [ ]:
scvi_model = scvi.model.SCVI(adata,
                            n_latent = 50, 
                            n_layers = 3, 
                            dispersion = 'gene-batch', 
                            gene_likelihood = 'nb')

In [ ]:
scvi_model.train(50, 
                 check_val_every_n_epoch = 1, 
                 enable_progress_bar = True, 
                 accelerator = "gpu",
                 devices = [0])

In [ ]:
adata.obsm["X_scVI"] = scvi_model.get_latent_representation(adata)

#### Evaluate model performance using the [_Svensson_](https://www.nxn.se/valent/2023/8/10/training-scvi-posterior-predictive-distributions-over-epochs) method

In [ ]:
history_df = (
    scvi_model.history['elbo_train'].astype(float)
    .join(scvi_model.history['elbo_validation'].astype(float))
    .reset_index()
    .melt(id_vars = ['epoch'])
)

p.options.figure_size = 12, 6

p_ = (
    p.ggplot(p.aes(x = 'epoch', y = 'value', color = 'variable'), history_df.query('epoch > 0'))
    + p.geom_line()
    + p.geom_point()
    + p.scale_color_manual({'elbo_train': 'black', 'elbo_validation': 'red'})
    + p.theme_minimal()
)

print(p_)

#### Label transfer with `scANVI` 

In [ ]:
scanvi_model = scvi.model.SCANVI.from_scvi_model(scvi_model, 'Unknown')

In [ ]:
scanvi_model.train(50, 
                   check_val_every_n_epoch = 1, 
                   enable_progress_bar = True,
                   accelerator = "gpu",
                   devices = [0])

In [ ]:
adata.obs["C_scANVI"] = scanvi_model.predict(adata)

- Extract latent representation

In [ ]:
adata.obsm["X_scANVI"] = scanvi_model.get_latent_representation(adata)

### Explore model performance using the [_Svensson_](https://www.nxn.se/valent/2023/8/10/training-scvi-posterior-predictive-distributions-over-epochs) method

In [ ]:
history_df = (
    scanvi_model.history['elbo_train'].astype(float)
    .join(scanvi_model.history['elbo_validation'].astype(float))
    .reset_index()
    .melt(id_vars = ['epoch'])
)

p.options.figure_size = 12, 6

p_ = (
    p.ggplot(p.aes(x = 'epoch', y = 'value', color = 'variable'), history_df.query('epoch > 0'))
    + p.geom_line()
    + p.geom_point()
    + p.scale_color_manual({'elbo_train': 'black', 'elbo_validation': 'red'})
    + p.theme_minimal()
)

p_.save('fig1.png', dpi = 300)

print(p_)

+ Visualize dataset

In [ ]:
sc.pp.neighbors(adata, use_rep = "X_scANVI", n_neighbors = 50, metric = 'minkowski')
sc.tl.umap(adata, min_dist = 0.2, spread = 2, random_state = 1712)

In [ ]:
adata.obs['C_scANVI'].value_counts()

In [ ]:
sc.set_figure_params(dpi = 300, figsize=(10,7))
sc.pl.umap(adata, color = ['Study_name', 'seed_labels', 'C_scANVI','Age_group', 'Library_Preparation_Protocol', 'Donor_ID'], ncols = 3, frameon = False, size = 5)

+ Export anndata object

In [ ]:
adata_filtered = adata[adata.obs['dataset'] == 'query']

In [ ]:
adata_raw = adata_raw[adata_raw.obs['dataset'] == 'query']

In [ ]:
adata_filtered.obs['C_scANVI'].value_counts()

In [ ]:
adata_filtered = anndata.AnnData(X = adata_raw.X, obs = adata_filtered.obs, var = adata_raw.var)

In [ ]:
adata_filtered.write_h5ad('/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/Adult_stem_cells/Adult_stem_cells.h5ad')

+ Check the scIB metrics

In [ ]:
from rich import print
import scib
import scib.metrics

In [ ]:
from scib_metrics.benchmark import Benchmarker

In [ ]:
bm = Benchmarker(
    adata,
    batch_key='Donor_ID',
    label_key='C_scANVI',
    embedding_obsm_keys=['X_scANVI', 'X_scVI', 'X_pca', 'X_umap'],
    n_jobs=-1
)

bm.benchmark()

In [ ]:
sc.set_figure_params(dpi = 300, figsize=(11,7))
bm.plot_results_table(min_max_scale=False)